In [54]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Ad-tech

In [55]:
data = pd.read_csv('data_adtech\AdSmartABdata - AdSmartABdata.csv')
data

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0
...,...,...,...,...,...,...,...,...,...
8072,ffea24ec-cec1-43fb-b1d1-8f93828c2be2,exposed,2020-07-05,7,Generic Smartphone,6,Chrome Mobile,0,0
8073,ffea3210-2c3e-426f-a77d-0aa72e73b20f,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0
8074,ffeaa0f1-1d72-4ba9-afb4-314b3b00a7c7,control,2020-07-04,9,Generic Smartphone,6,Chrome Mobile,0,0
8075,ffeeed62-3f7c-4a6e-8ba7-95d303d40969,exposed,2020-07-05,15,Samsung SM-A515F,6,Samsung Internet,0,0


## Generating pre-experiment AA data
to be used as one CUPED covariate

In [ ]:
np.random.seed(42)

mean_yes = data["yes"].mean()
var_yes = data["yes"].var()

desired_variance_reduction = 0.2  # (1 - 0.8)

# Compute the target variance for the covariate
target_variance = (1 - desired_variance_reduction) * var_yes

# Generate the CUPED covariate 'yes_aa'
yes_aa = (data["yes"] - mean_yes) + np.random.normal(0, np.sqrt(target_variance), len(data))
yes_aa = yes_aa.clip(0, 1).astype(int)  # Clip to ensure binary values

data["yes_aa"] = yes_aa

data.head()

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no,yes_aa
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1,0
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0,0


In [57]:
from scipy import stats

# Separate the data into control and experimental groups
control_group = data[data['experiment']=='control']['yes']
experimental_group = data[data['experiment']=='exposed']['yes']

# Perform an independent t-test
t_stat, p_value = stats.ttest_ind(control_group, experimental_group, equal_var=False)

# Display the results
t_stat, p_value

(-2.107278649715408, 0.035124460220699455)

In [58]:
p1 = control_group.sum() / data[data['experiment']=='control']['auction_id'].count()
p2 = experimental_group.sum() / data[data['experiment']=='exposed']['auction_id'].count()
print ('Conversion rate control: ', p1)
print ('Conversion rate treatment: ', p2)

Conversion rate control:  0.06484893146647015
Conversion rate treatment:  0.07688467299051423


In [59]:
p0 = data['yes'].mean()
print('Overall conversion: ', p0)
n = data['auction_id'].nunique()
var0 = p0*(1-p0)
print('Overall var: ', var0)

Overall conversion:  0.07081837315835088
Overall var:  0.06580313118155544


### Calculating minimal sample size

In [60]:
delta = p2-p1
za = 1.96
zb = 0.84
min_size = 2*(za+zb)**2 * var0 / delta**2
print('Minimal sample size: ', min_size)

Minimal sample size:  7122.73707263567


## CUPED
Try to find covariate. We first try to find device_make as covariate

In [61]:
# One-hot encoding of browser categories
browser_dummies = pd.get_dummies(data['device_make'], prefix='device', drop_first=True)
data = pd.concat([data, browser_dummies], axis=1)

# Compute theta for each dummy variable
theta = {}
for col in browser_dummies.columns:
    cov_y_x = np.cov(data['yes'], data[col])[0, 1]  # Covariance between data['yes'] and X_i
    var_x = np.var(data[col])  # Variance of X_i
    theta[col] = cov_y_x / var_x if var_x != 0 else 0  # Handle division by zero

# Initialize Y_adjusted as a copy of 'yes'
data['Y_adjusted'] = data['yes']

# Adjust Y for each browser type
for col in browser_dummies.columns:
    mean_x = data[col].mean()
    data['Y_adjusted'] -= theta[col] * (data[col] - mean_x)

print(theta)
data['Y_adjusted']

{'device_5099Y': -0.07083591222661575, 'device_6039Y': -0.07083591222661594, 'device_A0001': 0.42934108170522356, 'device_ALE-L21': -0.07083591222661595, 'device_ANE-LX1': 0.09607426812612772, 'device_ATU-L11': -0.07084468447580787, 'device_Armor_3': -0.07083591222661599, 'device_Asus I01WD': -0.07083591222661598, 'device_BBB100-2': -0.07083591222661592, 'device_BBF100-1': -0.07084468447580794, 'device_BKL-L09': -0.07085345889796255, 'device_BLA-L09': -0.07084468447580779, 'device_C2105': -0.07083591222661575, 'device_C6903': -0.07083591222661592, 'device_CLT-L09': -0.07105587220744632, 'device_CLT-L29': -0.07090615110834785, 'device_COL-L29': -0.07086223549388722, 'device_COR-L29': -0.07083591222661592, 'device_CPH1851': -0.07083591222661592, 'device_CUBOT_X18_Plus': -0.07083591222661575, 'device_D5503': -0.07083591222661587, 'device_D5803': -0.07083591222661594, 'device_D6503': -0.07083591222661592, 'device_D6603': -0.07084468447580779, 'device_DLI-L22': -0.07083591222661587, 'device

0       0.004874
1       0.004874
2       0.068112
3      -0.003348
4       0.004874
          ...   
8072    0.004874
8073    0.004874
8074    0.004874
8075    0.068288
8076   -0.036208
Name: Y_adjusted, Length: 8077, dtype: float64

In [62]:
var_cuped = data['Y_adjusted'].var()
print('Adjusted variance: ', var_cuped)

min_size = 2*(za+zb)**2 * var_cuped / delta**2
print('Minimal sample size: ', min_size)

Adjusted variance:  0.0635600111028979
Minimal sample size:  6879.934727887888


Also test if this covariate is homogeneous between the two groups

In [63]:
from scipy.stats import chi2_contingency
contingency_table = pd.crosstab(data['device_make'], data['experiment'])

# Step 2: Perform the Chi-Square Test of Independence
chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)

# Display the results
print("Chi-Square Statistic:", chi2_stat)
print("p-value:", p_value)
print("Degrees of Freedom:", dof)

Chi-Square Statistic: 453.2794894355725
p-value: 1.0878252726396005e-11
Degrees of Freedom: 268


In [64]:
data.head(1)

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no,yes_aa,...,device_XiaoMi Redmi Note 7,device_XiaoMi Redmi Note 8,device_XiaoMi Redmi Note 8 Pro,device_XiaoMi Redmi Note 8T,device_XiaoMi Redmi S2,device_YAL-L21,device_iPhone,device_moto e5 play,device_moto e5 plus,Y_adjusted
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0,0,...,False,False,False,False,False,False,False,False,False,0.004874


Now we choose yes_aa as covariate

In [65]:
cov_y_x = np.cov(data['yes'], data['yes_aa'])[0, 1]  # Covariance between data['yes'] and X_i
var_x = np.var(data['yes_aa'])  # Variance of X_i

# Initialize Y_adjusted as a copy of 'yes'
data['Y_adjusted2'] = data['yes']
theta = cov_y_x / var_x if var_x != 0 else 0

mean_x = data['yes_aa'].mean()
data['Y_adjusted2'] -= theta * (data['yes_aa'] - mean_x)

print(cov_y_x)
print(theta)

var_cuped = data['Y_adjusted2'].var()
print('Adjusted variance: ', var_cuped)

min_size = 2*(za+zb)**2 * var_cuped / delta**2
print('Minimal sample size: ', min_size)

0.02485181171344676
0.9548313569115278
Adjusted variance:  0.04208492831481391
Minimal sample size:  4555.404487973091


## PSM (to reweight features)

Actually I don't wanna do this.... What about let's generate AA test data with necessary associations.

In [66]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors

data_encoded = pd.get_dummies(data['device_make'], columns=['device_make'], drop_first=True)

X = data_encoded.drop(columns=['auction_id', 'experiment', 'date',  'yes'])  # Exclude experiment and target variable
y = data_encoded['experiment']

# Logistic regression for propensity score estimation
logistic = LogisticRegression(solver='liblinear')
logistic.fit(X, y)
data_encoded['propensity_score'] = logistic.predict_proba(X)[:, 1]  # Propensity scores

# Step 3: Matching
# Split the data into treatment and control groups
treated = data_encoded[data_encoded['experiment'] == 1]
control = data_encoded[data_encoded['experiment'] == 0]

# Use Nearest Neighbors to match control units to treated units based on propensity score
nn = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(control[['propensity_score']])
distances, indices = nn.kneighbors(treated[['propensity_score']])

# Get matched control indices
control_matched = control.iloc[indices.flatten()].reset_index(drop=True)
treated_matched = treated.reset_index(drop=True)

# Step 4: Combine matched samples
matched_data = pd.concat([treated_matched, control_matched])
matched_data

KeyError: "['auction_id', 'experiment', 'date', 'yes'] not found in axis"